In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submssion = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [5]:
data = pd.concat([train,test],sort=False)
data.info()

data['Sex'] = data['Sex'].map({'male':0, 'female':1})
data['Fare'] = data['Fare'].fillna(np.mean(data['Fare']))
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Embarked'] = data['Embarked'].map({'S':0, 'C':1,'Q':2})

In [16]:
delete_columns = ['Name','PassengerId','SibSp','Parch','Ticket','Cabin']
data.drop(delete_columns,axis=1,inplace=True)

In [19]:
train = data[:len(train)]
test = data[len(train):]

In [20]:
y_train = train['Survived']
X_train = train.drop('Survived',axis=1)
X_test = test.drop('Survived',axis=1)

In [22]:
#lightGBMを使用
X_train, X_valid,y_train,y_valid = \
    train_test_split(X_train,y_train,test_size=0.3,random_state=0,stratify=y_train)

categorical_features = ['Embarked','Pclass','Sex']

import lightgbm as lgb
#学習用データセット
lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
#評価用データセット
lgb_eval = lgb.Dataset(X_valid,y_valid,categorical_feature=categorical_features)

params = {
    'objctive':'binary'
}

In [26]:
from lightgbm import early_stopping, log_evaluation

verbose_eval = 10

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    callbacks=[
        early_stopping(stopping_rounds=10),  # 早期停止のコールバック
        log_evaluation(verbose_eval)           # ログ表示のコールバック
    ],
    num_boost_round=1000
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Unknown parameter: objctive
[LightGBM] [Warning] Unknown parameter: objctive
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002762 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 181
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 5
[LightGBM] [Warning] Unknown parameter: objctive
[LightGBM] [Info] Start training from score 0.383628
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

In [28]:
#ハイパーパラメータ調整
import optuna
from sklearn.metrics import log_loss

def objective(trial):
    params = {
        'objective':'binary',
        'max_bin':trial.suggest_int('num_bin',255,500),#探索範囲、項目設定
        'learning_rate':0.05,
        'num_leaves':trial.suggest_int('num_leaves',32,128),
    }
    #学習用データセット
    lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
    #評価用データセット
    lgb_eval = lgb.Dataset(X_valid,y_valid,categorical_feature=categorical_features)
    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        callbacks=[
            early_stopping(stopping_rounds=10),  # 早期停止のコールバック
            log_evaluation(verbose_eval)           # ログ表示のコールバック
        ],
        num_boost_round=1000
    )
    y_pred_valid = model.predict(X_valid,num_iteration=model.best_iteration)

    score = log_loss(y_valid,y_pred_valid)
    return score

In [29]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective,n_trials=40)#最小化したい関数と実行回数を指定

[I 2024-12-08 08:03:59,158] A new study created in memory with name: no-name-9cfb6359-5919-4ee0-9c70-9659f194bca7


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000783 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 181
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[I 2024-12-08 08:03:59,638] Trial 0 finished with value: 0.4402736570449865 and parameters: {'num_bin': 390, 'num_leaves': 101}. Best is trial 0 with value: 0.4402736570449865.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[80]	training's binary_logloss: 0.278628	valid_1's binary_logloss: 0.445379
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[72]	training's binary_logloss: 0.290827	valid_1's binary_logloss: 0.440274
[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead 

[I 2024-12-08 08:04:00,161] Trial 1 finished with value: 0.4402736570449865 and parameters: {'num_bin': 403, 'num_leaves': 84}. Best is trial 0 with value: 0.4402736570449865.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	training's binary_logloss: 0.294241	valid_1's binary_logloss: 0.440499
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

[I 2024-12-08 08:04:19,505] Trial 38 finished with value: 0.4402736570449865 and parameters: {'num_bin': 324, 'num_leaves': 43}. Best is trial 0 with value: 0.4402736570449865.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	training's binary_logloss: 0.312621	valid_1's binary_logloss: 0.443413
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

[I 2024-12-08 08:04:19,961] Trial 39 finished with value: 0.4402736570449865 and parameters: {'num_bin': 327, 'num_leaves': 43}. Best is trial 0 with value: 0.4402736570449865.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	training's binary_logloss: 0.332445	valid_1's binary_logloss: 0.449511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [30]:
#最適な値
study.best_params

{'num_bin': 390, 'num_leaves': 101}

In [33]:
#以降best_paramsでモデル作成
#クロスバリデーションを実装
from sklearn.model_selection import KFold

y_preds = []#X_testに対する予測値を格納
models = []#モデルを格納
off_train = np.zeros((len(X_train),))#X_valに対する予測値を格納
cv = KFold(n_splits=5,shuffle=True,random_state=0)

params = {
        'objective':'binary',
        'max_bin':390,
        'learning_rate':0.05,
        'num_leaves':101,
    }

for fold_id,(train_index,valid_index) in enumerate(cv.split(X_train)):
    # データの分割
    X_tr = X_train.iloc[train_index, :]
    X_val = X_train.iloc[valid_index, :]
    y_tr = y_train.iloc[train_index]
    y_val = y_train.iloc[valid_index]
    
    lgb_train = lgb.Dataset(X_tr,y_tr,categorical_feature = categorical_features)
    lgb_eval = lgb.Dataset(X_val,y_val,categorical_feature = categorical_features)

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        callbacks=[
            early_stopping(stopping_rounds=10),  # 早期停止のコールバック
            log_evaluation(verbose_eval)           # ログ表示のコールバック
        ],
        num_boost_round=1000
    )

    off_train[valid_index] = model.predict(X_val,num_iteration = model.best_iteration)
    y_pred = model.predict(X_test,num_iteration = model.best_iteration)
    y_preds.append(y_pred)
    models.append(model)

[LightGBM] [Info] Number of positive: 195, number of negative: 303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000687 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 163
[LightGBM] [Info] Number of data points in the train set: 498, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391566 -> initscore=-0.440733
[LightGBM] [Info] Start training from score -0.440733
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

In [37]:
y_sub = np.mean(y_preds, axis=0)#平均を計算
y_sub = (y_sub > 0.5).astype(int)#0/1に変換

In [38]:
print(y_sub)#結果

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
